In [6]:
# preliminaries
import sys,os,time,cv2,json
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as io
import ops
from moviepy.editor import *

# for video file generation
from utils import imread_to_rgb, crop_img

DB_PATH = '/home/jhchoi/datasets2/friends/'
MD_PATH = '../../anno/'
window_size = 4
crop_size = ops.img_sz


In [2]:
def hmsf_to_fnum(hmsf, fps):
    hms = hmsf.split(';')[0].split(':')
    f   = hmsf.split(';')[1]
    
    return (int(hms[0])*60*60 + int(hms[1])*60 + int(hms[2]))*fps + int(f)

def emo_char_idx(emo):
    # 0=angry, 1=disgust, 2=fear, 3=happy, 4=sad, 5=surprise, 6=neutral
    if emo == 'angry' or emo == 'anger':
        return 0
    elif emo == 'disgust':
        return 1
    elif emo == 'fear':
        return 2
    elif emo == 'happy' or emo == 'happiness':
        return 3
    elif emo == 'sad' or emo == 'sadness':
        return 4
    elif emo == 'surprise':
        return 5
    elif emo == 'neutral':
        return 6
    else:
        'error'

def coor_change(coor):
    spl = coor.split('  ')
    if len(spl)>1:
        return int(spl[0])
    else:
        return int(coor)

    
epi_file = {'ep01': '1x01 - The One Where Monica Gets A Roommate.mkv',
            'ep02': '1x02 - The One With The Sonogram At The End.mkv',
            'ep03': '1x03 - The One With The Thumb.mkv',
            'ep04': '1x04 - The One With George Stephanopoulos.mkv',
            'ep05': '1x05 - The One With The East German Laundry Detergent.mkv',
            'ep06': '1x06 - The One With The Butt.mkv',
            'ep07': '1x07 - The One With The Blackout.mkv',
            'ep08': '1x08 - The One Where Nana Dies Twice.mkv',
            'ep09': '1x09 - The One Where Underdog Gets Away.mkv',
            'ep10': '1x10 - The One With The Monkey.mkv'
           }


In [5]:
epi_list = ['ep01', 'ep02', 'ep03', 'ep04', 'ep05', 'ep06', 'ep07', 'ep08', 'ep09', 'ep10']
val_dict = dict()
numl_all = np.zeros(7)

for i in range(7):
    val_dict[i] = dict()

# for every episode
for epi in epi_list:
    print '\n'+epi
    if not os.path.exists(os.path.join(DB_PATH, epi+'_p_img')):
        os.mkdir(os.path.join(DB_PATH, epi+'_p_img'))
    
    # open 
    with open(MD_PATH+'s01_'+epi+'_tag2_visual_Final_180809.json') as md:
        epi_md = json.load(md)

    epi_md = epi_md['visual_results']

    # every period
    for i in range(len(epi_md)):
        sys.stdout.write("\r"+str(i+1)+'/'+str(len(epi_md)))
        # per num
        pnum = int(epi_md[i]['period_num'])
        
        # start-end
        stime = hmsf_to_fnum(epi_md[i]['start_time'], 24)
        etime = hmsf_to_fnum(epi_md[i]['end_time'], 24)
        # img
        imname = epi_md[i]['image']
        if imname[0:2] == 'Fr':
            pfnum = int(epi_md[i]['image'].split('.')[0].split('_')[1])
        else:
            pfnum = int(epi_md[i]['image'].split('.')[0][1:])
        # person
        pid_md = epi_md[i]['person'][0]

        cidx = 0
        # for every person
        for char in pid_md.keys():
            emo = pid_md[char][0]['emotion'].lower()
            face_bb = pid_md[char][0]['face_rect']

            if emo == 'none' or face_bb['max_x'] == 'none':
                continue

            # face xy
            face_bb = [coor_change(face_bb['min_x']), coor_change(face_bb['min_y']), coor_change(face_bb['max_x']), coor_change(face_bb['max_y'])]
            
            if face_bb[0] >= face_bb[2] or face_bb[1] >= face_bb[3]:
                continue
                
            emo_idx = emo_char_idx(emo)
            bat_img_temp = list() 
            
            for i in range(window_size):
                pfnum_batch = pfnum-window_size/2+i
                # fimg
                frame_img = cv2.imread(DB_PATH+'/'+ epi +'/'+str('%05d' % pfnum_batch)+'.jpg')
                
                face_w = face_bb[2] - face_bb[0]
                face_h = face_bb[3] - face_bb[1]
                face_m = (face_w+face_h)*0.3
                face_s = int(np.sqrt((face_w + face_m)*(face_h + face_m)))
                
                face_crop = crop_img(frame_img, int(face_bb[0]+face_w/2.), int(face_bb[1]+face_h/2.), face_s, face_s, True)
                face_resz = cv2.resize(face_crop, (ops.img_sz,ops.img_sz))
                savename = epi + '_p_img/' + str('%04d' % pnum)+'_'+char+'_'+str(emo_idx)+'_'+str('%05d' % pfnum_batch)+'.jpg'
                cv2.imwrite(DB_PATH+savename, face_resz)
                bat_img_temp.append(savename)

            # save to dict
            numl = numl_all[emo_idx]
            val_dict[emo_idx][numl] = dict()
            val_dict[emo_idx][numl]['img'] = bat_img_temp
            val_dict[emo_idx][numl]['emo'] = emo_idx
            val_dict[emo_idx][numl]['pos'] = [epi, pnum, char, pfnum]
            numl_all[emo_idx] += 1




ep01
1152/1152
ep02
1398/1398
ep03
1221/1221
ep04
1071/1071
ep05
855/855
ep06
840/840
ep07
639/639
ep08
744/744
ep09
768/768
ep10
816/816

In [9]:
# extract audio files
for emo in val_dict.keys():
    print '\n'+str(emo)
    for i,ins in enumerate(val_dict[emo].keys()):
        sys.stdout.write("\r"+str(i+1)+'/'+str(len(val_dict[emo].keys())))
        
        epi, pnum, char, fnum = val_dict[emo][ins]['pos']
        raw_audio = AudioFileClip(os.path.join(DB_PATH,epi_file[epi]))
        sec = fnum/24.
        crop_audio = raw_audio.subclip(int(sec-0.5), int(sec-0.5)+1)
        
        filename = str('%04d' % pnum)+'_'+char+'_'+str(emo)+'_'+str('%05d' % fnum)+'.wav'
        save_path = os.path.join(DB_PATH, epi+'_p_wav', filename)
        
        val_dict[emo_idx][numl]['wav'] = filename
        
        if not os.path.exists(os.path.join(DB_PATH, epi+'_p_wav')):
            os.mkdir(os.path.join(DB_PATH, epi+'_p_wav'))
        
        crop_audio.write_audiofile(save_path, logger=None)
        


0
633/633
1
646/646
2
145/145
3
2728/2728
4
476/476
5
622/622
6
8119/8119

In [10]:
np.save('../../dicts/friends_parsed_new.npy', val_dict)

In [ ]:
from scipy.io import wavfile
fs, data = wavfile.read('test.wav')